This notebook will concern getting the data from spotify API (some parts here are hardcoded for the extraction of some data)

Spotify Link got from this dataset : https://www.kaggle.com/datasets/mrmorj/dataset-of-songs-in-spotify?select=playlists.csv

## Reading at the dataset and extracting the dataset

In [7]:
import numpy as np
import pandas as pd
import requests
import time

In [3]:
orig_dataset = pd.read_csv('data/dataset.csv', low_memory = False)

In [4]:
len(orig_dataset)

42305

In [5]:
orig_dataset.shape

(42305, 22)

In [6]:
orig_dataset.head(5)

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,id,uri,track_href,analysis_url,duration_ms,time_signature,genre,song_name,Unnamed: 0,title
0,0.831,0.814,2,-7.364,1,0.4200,0.0598,0.013400,0.0556,0.3890,...,2Vc6NJ9PW9gD9q343XFRKx,spotify:track:2Vc6NJ9PW9gD9q343XFRKx,https://api.spotify.com/v1/tracks/2Vc6NJ9PW9gD...,https://api.spotify.com/v1/audio-analysis/2Vc6...,124539,4,Dark Trap,Mercury: Retrograde,NaN,NaN
1,0.719,0.493,8,-7.230,1,0.0794,0.4010,0.000000,0.1180,0.1240,...,7pgJBLVz5VmnL7uGHmRj6p,spotify:track:7pgJBLVz5VmnL7uGHmRj6p,https://api.spotify.com/v1/tracks/7pgJBLVz5Vmn...,https://api.spotify.com/v1/audio-analysis/7pgJ...,224427,4,Dark Trap,Pathology,NaN,NaN
2,0.850,0.893,5,-4.783,1,0.0623,0.0138,0.000004,0.3720,0.0391,...,0vSWgAlfpye0WCGeNmuNhy,spotify:track:0vSWgAlfpye0WCGeNmuNhy,https://api.spotify.com/v1/tracks/0vSWgAlfpye0...,https://api.spotify.com/v1/audio-analysis/0vSW...,98821,4,Dark Trap,Symbiote,NaN,NaN
3,0.476,0.781,0,-4.710,1,0.1030,0.0237,0.000000,0.1140,0.1750,...,0VSXnJqQkwuH2ei1nOQ1nu,spotify:track:0VSXnJqQkwuH2ei1nOQ1nu,https://api.spotify.com/v1/tracks/0VSXnJqQkwuH...,https://api.spotify.com/v1/audio-analysis/0VSX...,123661,3,Dark Trap,ProductOfDrugs (Prod. The Virus and Antidote),NaN,NaN
4,0.798,0.624,2,-7.668,1,0.2930,0.2170,0.000000,0.1660,0.5910,...,4jCeguq9rMTlbMmPHuO7S3,spotify:track:4jCeguq9rMTlbMmPHuO7S3,https://api.spotify.com/v1/tracks/4jCeguq9rMTl...,https://api.spotify.com/v1/audio-analysis/4jCe...,123298,4,Dark Trap,Venom,NaN,NaN


In [7]:
orig_dataset.columns

Index(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms',
       'time_signature', 'genre', 'song_name', 'Unnamed: 0', 'title'],
      dtype='object')

In [8]:
#this uri redirects you to the song straight away
print("Direct you to song: \n", orig_dataset['uri'].iloc[1])
print('song name',orig_dataset['song_name'].iloc[1] )
print("\n")

# this URL will contain some of the metadata that isn't available straight from the dataset
print("this URL will contain some of the metadata that isn't available straight from the dataset:\n"
      , orig_dataset['track_href'].iloc[0])
print("\n")

print("This will have all the audio analysis in the song:\n ", orig_dataset['analysis_url'].iloc[0])

Direct you to song: 
 spotify:track:7pgJBLVz5VmnL7uGHmRj6p
song name Pathology


this URL will contain some of the metadata that isn't available straight from the dataset:
 https://api.spotify.com/v1/tracks/2Vc6NJ9PW9gD9q343XFRKx


This will have all the audio analysis in the song:
  https://api.spotify.com/v1/audio-analysis/2Vc6NJ9PW9gD9q343XFRKx


## Reading from Spotify API

a lot of tries were made to fetch data from the spotify app fast without reaching the rate fetching limit and eventually "blocked"

Dataset does not have alot of the metadata that is contained within these href links, e.g. date released, artist name, etc. therefore these metadata will be extracted and a new dataframe would be created with these enhancements
From initial analysis.

We should also extract the artist name, release_date , explicitness, and popularity of each track from the link and add them to our new dataframe.

In [9]:
# Spotify access token
# if access token lost use this
# https://accounts.spotify.com/authorize?client_id=402ca467b1554a058be52a2e0659ff1d&response_type=token&redirect_uri=http://localhost:8888/callback&scope=user-library-read
access_token = 'BQAUj2-OTOZNVqHrwoRT4OKgVGhS9IhGXwdL5WQ7gscjGTHMvbQWYX2SSio9gHEwLhkDJQBWkd1IYPFqgMEiauQtdXHuy1HhFffz7vrqzSkQfNrAzMHdIfh5L6EgZsCDd97UtvQFY8mNUTFyvHs-xSyTEBhCqno_CXQLC2LP8W896igvn1HeI2ADBron48W-Hw0PEUNDuhuEKT7pZn7Y'
headers = {"Authorization": f"Bearer {access_token}"}

In [10]:
# test to see that the API works and the access_token work
track_href_link = 'https://api.spotify.com/v1/tracks/7pgJBLVz5VmnL7uGHmRj6p'
track_href_response = requests.get(track_href_link, headers=headers)
track_href_response
track_metadata = track_href_response.json()
track_metadata['name']

'Pathology'

## This is for the track metadata requests

### This involves a NON-concurrent batch fetching

In [ ]:
# import requests
# import time

# # Define the rate limit and backoff parameters
# REQUESTS_PER_SECOND = 2  # Example rate limit: 4 requests per second
# BACKOFF_TIME = 2  # Seconds to wait if rate limit is hit

# tracks_metadata = []

# def fetch_data_batch(batch_urls):
#     batch_responses = []
#     for url in batch_urls:
#         response = requests.get(url, headers=headers)
        
#         # Check if rate limit exceeded
#         while response.status_code == 429:
#             retry_after = int(response.headers.get('Retry-After', BACKOFF_TIME))
#             print("Rate limit exceeded. Waiting for", retry_after, "seconds...")
#             time.sleep(retry_after)
#             response = requests.get(url, headers=headers)
        
#         if response.ok:
#             batch_responses.append(response.json())
        
#         # Add a delay to respect rate limits
#         time.sleep(1 / REQUESTS_PER_SECOND)
    
#     return batch_responses

# # Fetch data in batches
# batch_size = 5  # Adjust as needed
# num_batches = (len(urls) + batch_size - 1) // batch_size

# for batch_index in range(num_batches):
#     batch_start = batch_index * batch_size
#     batch_end = min((batch_index + 1) * batch_size, len(urls))
#     batch_urls = urls[batch_start:batch_end]
    
#     batch_responses = fetch_data_batch(batch_urls)
#     tracks_metadata.extend(batch_responses)
    
#     # Print progress
#     total_processed = batch_index * batch_size + len(batch_responses)
#     print("Processed", total_processed, "out of", len(urls), "requests")
#     if (total_processed % csv_split) == 0:
#         # Generate a dynamic filename with timestamp
#         dummy = []
#         print("Total Processed saved as of now: ", total_processed)
#         print("length of data saved", len(tracks_metadata))
#         dummy = pd.DataFrame(tracks_metadata)
#         timestamp = time.strftime('%Y%m%d_%H%M%S')
#         dynamic_filename = f'data_{timestamp}.csv'

#         # Save DataFrame to the dynamic filename
#         dummy.to_csv(dynamic_filename, index=False)
        

### This involves concurrent threading

<br>
Since we will be making lots of requests to Spotify's API to collect these data from the links in the dataset, which can be very time consuming. We will be fetching the data concurrently using the concurrent.future.ThreadPoolExecuter method. However, this method can definitely exceed the API's requests rate limit so we will be implementing also a rate-limiting and a backing off mechanism to the api so that we do not get blocked off from sending too many requests.

Noticed that spotify api only receives 10,000 requests a day per app on an account, therefore i will be making 10 excel files for each type of link (track_href_link and analysis_url_link) and in the end they will be combined making two large csv files which i can then filter from. that is why this part is **hardcoded** as it was for the retrieval of the rest of the data

In [ ]:
len(orig_dataset)

In [ ]:
csv_split*9

In [ ]:
csv_split = 4500

# List of URLs to fetch
tracks_metadata_urls = orig_dataset['track_href'].iloc[csv_split*9:csv_split*10]

print(len(tracks_metadata_urls))

print("first song fourth split:", orig_dataset['song_name'].iloc[csv_split*8])
print("last song fourth split:", orig_dataset['song_name'].iloc[(csv_split*9)-1])

print("first song fourth split:", orig_dataset['song_name'].iloc[csv_split*9])
#print("last song fourth split:", orig_dataset['song_name'].iloc[(csv_split*10)-1])

In [ ]:
li1 = orig_dataset['track_href'].iloc[csv_split*9]
a = li1
b = requests.get(a, headers=headers)
c = b.json()
print("first song in this split: ", c['name'])

li2 = orig_dataset['track_href'].iloc[len(orig_dataset)-1]
a = li2
b = requests.get(a, headers=headers)
c = b.json()
print("last song in this split: ", c['name'])

In [ ]:
import concurrent.futures
import requests
import time

# Define the rate limit and backoff parameters
REQUESTS_PER_SECOND = 5  # Example rate limit: 4 requests per second
BACKOFF_TIME = 3  # Seconds to wait if rate limit is hit


tracks_metadata = []
total_processed = 0


def fetch_data(url):
    try:
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code == 429:
            retry_after = int(response.headers.get('Retry-After', BACKOFF_TIME))
            print("Rate limit exceeded. Waiting for", retry_after, "seconds...")
            time.sleep(retry_after)
            return fetch_data(url)  # Retry request after waiting
        elif response.status_code == 200:
            return response.json()
        
        else:
            print("Error:", response.status_code)
            return None
        
    except requests.exceptions.Timeout:
        
        print("Request timed out. Retrying...")
        time.sleep(BACKOFF_TIME)
        return fetch_data(url)
    
    except Exception as e:
        print("An error occurred:", e)
        return None

# Function to process a batch of URLs and update progress
def process_batch(batch_urls, batch_index):
    global total_processed
    batch_responses = []
    for index, url in enumerate(batch_urls):
        response = fetch_data(url)
        if response is not None:
            batch_responses.append(response)
            
            # Update progress
            total_processed += 1
            print("Processed", total_processed, "out of", len(tracks_metadata_urls), "requests")
            
            # Add a delay to respect rate limits
            time.sleep(1 / REQUESTS_PER_SECOND)
    
    return batch_responses



# Use concurrent fetching with progress tracking
batch_size = 3  # Adjust as needed
num_batches = (len(tracks_metadata_urls) + batch_size - 1) // batch_size



with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    futures = []
    for batch_index in range(num_batches):
        batch_start = batch_index * batch_size
        batch_end = min((batch_index + 1) * batch_size, len(tracks_metadata_urls))
        batch_urls = tracks_metadata_urls[batch_start:batch_end]
        
        future = executor.submit(process_batch, batch_urls, batch_index)
        futures.append(future)
    
    
    # Wait for all futures to complete
    for future in concurrent.futures.as_completed(futures):
        batch_responses = future.result()
        tracks_metadata.extend(batch_responses)
        
        
print("Total processed:", total_processed)

In [ ]:
dummy = []
dummy = pd.DataFrame(tracks_metadata)

In [ ]:
dummy['name'].iloc[0:50]

In [ ]:
dummy['name'].iloc[1780:]

In [ ]:
timestamp = time.strftime('%Y%m%d_%H%M%S')
dynamic_filename = f'track_meta_data_{timestamp}.csv'
# Save DataFrame to the dynamic filename
dummy.to_csv(dynamic_filename, index=False)

<br>
Combining the csv files together and saving it

In [9]:
# Initialize an empty DataFrame to store the concatenated data
combined_df = pd.DataFrame()

# Loop through each CSV file and concatenate them
for i in range(1,11):
    df = pd.read_csv(f'track_meta_data_{i}.csv')
    combined_df = pd.concat([combined_df, df], ignore_index=True)


In [11]:
combined_df.to_csv('data/meta_dataset.csv', index= False)

<br> <br>

## This is for the audio analysis requests

we want to to extract the audio analysis of the songs from the spotify api. However, the problem with concurrent-threading-pool -executing is that it does not store the results in the same order, and the audio analysis responses of the songs do not contain the name or the id of the songs, so all the audio anaylsis links would be pointless since we cannot map it to the songs later on. Therefore, we can tweak the api response by including the song ID as an extra field within the API response itself, acting as if that is the response coming from the api, ensuring that with every audio analysis data, it has its own link with it as a key. This way, you don't need to create a separate mapping. To achieve this, you can modify the fetch_data function to include the song ID in the API response.

In [12]:
i = 3
csv_split = 9000

In [37]:
csv_split*(5)

45000

In [39]:
# List of URLs to fetch
audio_analysis_urls = orig_dataset['analysis_url'].iloc[csv_split*(4)+4500:]
# List of song IDs and URLs to fetch
song_data = [(song_id, url) for song_id, url in zip(orig_dataset['id'].iloc[csv_split*(4)+4500:],
                                                    orig_dataset['analysis_url'].iloc[csv_split*(4)+4500:])]

In [13]:
csv_split*(3-1) , (csv_split*(3-1)+4500)

(18000, 22500)

In [14]:
(csv_split*(3-1)+4500), csv_split*(3)

(22500, 27000)

In [15]:
csv_split*(3), csv_split*(3)+4500

(27000, 31500)

In [16]:
csv_split*(3)+4500 , csv_split*(4)

(31500, 36000)

In [17]:
csv_split*(4) , csv_split*(4)+4500

(36000, 40500)

In [38]:
csv_split*(4)+4500 , len(orig_dataset)

(40500, 42305)

In [40]:
print(len(audio_analysis_urls), len(song_data))

1805 1805


In [41]:
import concurrent.futures
import requests
import time

# Define the rate limit and backoff parameters
REQUESTS_PER_SECOND = 4  # Example rate limit: 4 requests per second
BACKOFF_TIME = 4  # Seconds to wait if rate limit is hit


tracks_audio_analysis = []
total_processed = 0


def fetch_data(song_id, url):
    try:
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code == 429:
            retry_after = int(response.headers.get('Retry-After', BACKOFF_TIME))
            print("Rate limit exceeded. Waiting for", retry_after, "seconds...")
            time.sleep(retry_after)
            return fetch_data(song_id,url)  # Retry request after waiting
        elif response.status_code == 200:
            json_response = response.json()
            json_response['_song_id'] = song_id  # Add song ID to the response
            return json_response
        
        else:
            print("Error:", response.status_code)
            print("with url:", song_id)
            return None
        
    except requests.exceptions.Timeout:
        
        print("Request timed out. Retrying...")
        time.sleep(BACKOFF_TIME)
        return fetch_data(song_id, url)
    
    except Exception as e:
        print("An error occurred:", e)
        print("with url:", song_id)
        return None
    
    
def process_batch(batch_data, batch_index):
    global total_processed
    batch_responses = []
    for song_id, url in batch_data:
        response = fetch_data(song_id, url)
        if response is not None:
            batch_responses.append(response)
            
            # Update progress
            total_processed += 1
            print("Processed", total_processed, "out of", len(audio_analysis_urls), "requests")
            
            # Add a delay to respect rate limits
            time.sleep(1 / REQUESTS_PER_SECOND)
    
    return batch_responses

# Use concurrent fetching with progress tracking
batch_size = 3  # Adjust as needed
num_batches = (len(song_data) + batch_size - 1) // batch_size

with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    futures = []
    for batch_index in range(num_batches):
        batch_start = batch_index * batch_size
        batch_end = min((batch_index + 1) * batch_size, len(song_data))
        batch_data = song_data[batch_start:batch_end]
        
        future = executor.submit(process_batch, batch_data, batch_index)
        futures.append(future)
    
    # Wait for all futures to complete
    for future in concurrent.futures.as_completed(futures):
        batch_responses = future.result()
        tracks_audio_analysis.extend(batch_responses)

print("Total processed:", total_processed)

Processed 1 out of 1805 requests
Processed 2 out of 1805 requests
Processed 3 out of 1805 requests
Processed 4 out of 1805 requests
Processed 5 out of 1805 requests
Processed 6 out of 1805 requests
Processed 7 out of 1805 requests
Processed 8 out of 1805 requests
Processed 9 out of 1805 requests
Processed 10 out of 1805 requests
Processed 11 out of 1805 requests
Processed 12 out of 1805 requests
Processed 13 out of 1805 requests
Processed 14 out of 1805 requests
Processed 15 out of 1805 requests
Processed 16 out of 1805 requests
Processed 17 out of 1805 requests
Processed 18 out of 1805 requests
Processed 19 out of 1805 requests
Processed 20 out of 1805 requests
Processed 21 out of 1805 requests
Processed 22 out of 1805 requests
Processed 23 out of 1805 requests
Processed 24 out of 1805 requests
Processed 25 out of 1805 requests
Processed 26 out of 1805 requests
Processed 27 out of 1805 requests
Processed 28 out of 1805 requests
Processed 29 out of 1805 requests
Processed 30 out of 180

Processed 241 out of 1805 requests
Processed 242 out of 1805 requests
Processed 243 out of 1805 requests
Processed 244 out of 1805 requests
Processed 245 out of 1805 requests
Processed 246 out of 1805 requests
Processed 247 out of 1805 requests
Processed 248 out of 1805 requests
Processed 249 out of 1805 requests
Processed 250 out of 1805 requests
Processed 251 out of 1805 requests
Processed 252 out of 1805 requests
Processed 253 out of 1805 requests
Processed 254 out of 1805 requests
Processed 255 out of 1805 requests
Processed 256 out of 1805 requests
Processed 257 out of 1805 requests
Processed 258 out of 1805 requests
Processed 259 out of 1805 requests
Processed 260 out of 1805 requests
Processed 261 out of 1805 requests
Processed 262 out of 1805 requests
Processed 263 out of 1805 requests
Processed 264 out of 1805 requests
Processed 265 out of 1805 requests
Processed 266 out of 1805 requests
Processed 267 out of 1805 requests
Processed 268 out of 1805 requests
Processed 269 out of

Processed 464 out of 1805 requests
Processed 465 out of 1805 requests
Processed 466 out of 1805 requests
Processed 467 out of 1805 requests
Processed 468 out of 1805 requests
Processed 469 out of 1805 requests
Processed 470 out of 1805 requests
Processed 471 out of 1805 requests
Processed 472 out of 1805 requests
Processed 473 out of 1805 requests
Processed 474 out of 1805 requests
Processed 475 out of 1805 requests
Processed 476 out of 1805 requests
Processed 477 out of 1805 requests
Processed 478 out of 1805 requests
Processed 479 out of 1805 requests
Processed 480 out of 1805 requests
Processed 481 out of 1805 requests
Processed 482 out of 1805 requests
Processed 483 out of 1805 requests
Processed 484 out of 1805 requests
Processed 485 out of 1805 requests
Processed 486 out of 1805 requests
Processed 487 out of 1805 requests
Processed 488 out of 1805 requests
Processed 489 out of 1805 requests
Processed 490 out of 1805 requests
Rate limit exceeded. Waiting for 1 seconds...
Rate limi

Processed 676 out of 1805 requests
Processed 677 out of 1805 requests
Processed 678 out of 1805 requests
Processed 679 out of 1805 requests
Processed 680 out of 1805 requests
Processed 681 out of 1805 requests
Processed 682 out of 1805 requests
Processed 683 out of 1805 requests
Processed 684 out of 1805 requests
Processed 685 out of 1805 requests
Processed 686 out of 1805 requests
Processed 687 out of 1805 requests
Processed 688 out of 1805 requests
Processed 689 out of 1805 requests
Processed 690 out of 1805 requests
Processed 691 out of 1805 requests
Processed 692 out of 1805 requests
Processed 693 out of 1805 requests
Rate limit exceeded. Waiting for 1 seconds...
Processed 694 out of 1805 requests
Rate limit exceeded. Waiting for 1 seconds...
Processed 695 out of 1805 requests
Rate limit exceeded. Waiting for 1 seconds...
Processed 696 out of 1805 requests
Processed 697 out of 1805 requests
Processed 698 out of 1805 requests
Processed 699 out of 1805 requests
Processed 700 out of 1

Processed 908 out of 1805 requests
Processed 909 out of 1805 requests
Processed 910 out of 1805 requests
Processed 911 out of 1805 requests
Processed 912 out of 1805 requests
Processed 913 out of 1805 requests
Processed 914 out of 1805 requests
Processed 915 out of 1805 requests
Processed 916 out of 1805 requests
Processed 917 out of 1805 requests
Processed 918 out of 1805 requests
Processed 919 out of 1805 requests
Processed 920 out of 1805 requests
Processed 921 out of 1805 requests
Processed 922 out of 1805 requests
Processed 923 out of 1805 requests
Processed 924 out of 1805 requests
Processed 925 out of 1805 requests
Processed 926 out of 1805 requests
Processed 927 out of 1805 requests
Processed 928 out of 1805 requests
Processed 929 out of 1805 requests
Processed 930 out of 1805 requests
Processed 931 out of 1805 requests
Processed 932 out of 1805 requests
Processed 933 out of 1805 requests
Processed 934 out of 1805 requests
Processed 935 out of 1805 requests
Processed 936 out of

Processed 1141 out of 1805 requests
Processed 1142 out of 1805 requests
Processed 1143 out of 1805 requests
Processed 1144 out of 1805 requests
Processed 1145 out of 1805 requests
Processed 1146 out of 1805 requests
Processed 1147 out of 1805 requests
Processed 1148 out of 1805 requests
Processed 1149 out of 1805 requests
Processed 1150 out of 1805 requests
Processed 1151 out of 1805 requests
Processed 1152 out of 1805 requests
Processed 1153 out of 1805 requests
Processed 1154 out of 1805 requests
Processed 1155 out of 1805 requests
Processed 1156 out of 1805 requests
Processed 1157 out of 1805 requests
Processed 1158 out of 1805 requests
Processed 1159 out of 1805 requests
Processed 1160 out of 1805 requests
Processed 1161 out of 1805 requests
Processed 1162 out of 1805 requests
Processed 1163 out of 1805 requests
Processed 1164 out of 1805 requests
Processed 1165 out of 1805 requests
Processed 1166 out of 1805 requests
Processed 1167 out of 1805 requests
Processed 1168 out of 1805 r

Processed 1359 out of 1805 requests
Processed 1360 out of 1805 requests
Processed 1361 out of 1805 requests
Processed 1362 out of 1805 requests
Processed 1363 out of 1805 requests
Processed 1364 out of 1805 requests
Processed 1365 out of 1805 requests
Processed 1366 out of 1805 requests
Processed 1367 out of 1805 requests
Processed 1368 out of 1805 requests
Processed 1369 out of 1805 requests
Processed 1370 out of 1805 requests
Processed 1371 out of 1805 requests
Processed 1372 out of 1805 requests
Processed 1373 out of 1805 requests
Processed 1374 out of 1805 requests
Processed 1375 out of 1805 requests
Processed 1376 out of 1805 requests
Processed 1377 out of 1805 requests
Processed 1378 out of 1805 requests
Processed 1379 out of 1805 requests
Processed 1380 out of 1805 requests
Processed 1381 out of 1805 requests
Processed 1382 out of 1805 requests
Processed 1383 out of 1805 requests
Processed 1384 out of 1805 requests
Processed 1385 out of 1805 requests
Processed 1386 out of 1805 r

Processed 1583 out of 1805 requests
Processed 1584 out of 1805 requests
Processed 1585 out of 1805 requests
Processed 1586 out of 1805 requests
Processed 1587 out of 1805 requests
Processed 1588 out of 1805 requests
Processed 1589 out of 1805 requests
Processed 1590 out of 1805 requests
Processed 1591 out of 1805 requests
Processed 1592 out of 1805 requests
Processed 1593 out of 1805 requests
Processed 1594 out of 1805 requests
Processed 1595 out of 1805 requests
Processed 1596 out of 1805 requests
Processed 1597 out of 1805 requests
Processed 1598 out of 1805 requests
Processed 1599 out of 1805 requests
Processed 1600 out of 1805 requests
Processed 1601 out of 1805 requests
Processed 1602 out of 1805 requests
Processed 1603 out of 1805 requests
Processed 1604 out of 1805 requests
Processed 1605 out of 1805 requests
Processed 1606 out of 1805 requests
Processed 1607 out of 1805 requests
Processed 1608 out of 1805 requests
Processed 1609 out of 1805 requests
Processed 1610 out of 1805 r

Processed 1798 out of 1805 requests
Processed 1799 out of 1805 requests
Processed 1800 out of 1805 requests
Processed 1801 out of 1805 requests
Processed 1802 out of 1805 requests
Processed 1803 out of 1805 requests
Processed 1804 out of 1805 requests
Processed 1805 out of 1805 requests
Total processed: 1805


In [42]:
len(tracks_audio_analysis)

1805

In [43]:
ids_that_went_in = [song_data[i][0] for i in range(len(song_data))]
ids_to_be_stored = [tracks_audio_analysis[i]['_song_id'] for i in range(len(tracks_audio_analysis))]


In [44]:
def find_duplicate_indexes(lst):
    index_dict = {}
    duplicate_indexes = []

    for index, item in enumerate(lst):
        if item in index_dict:
            duplicate_indexes.append((index, index_dict[item]))
        else:
            index_dict[item] = index

    return duplicate_indexes

duplicate_indexes = find_duplicate_indexes(ids_to_be_stored)
print("Total Duplicates: ", len(duplicate_indexes))
print("\n\n")

for idx1, idx2 in duplicate_indexes:
    print(f"Duplicates found at indexes {idx1} and {idx2}")

Total Duplicates:  267



Duplicates found at indexes 38 and 32
Duplicates found at indexes 74 and 43
Duplicates found at indexes 101 and 50
Duplicates found at indexes 166 and 34
Duplicates found at indexes 193 and 178
Duplicates found at indexes 217 and 106
Duplicates found at indexes 267 and 151
Duplicates found at indexes 272 and 201
Duplicates found at indexes 274 and 201
Duplicates found at indexes 298 and 138
Duplicates found at indexes 305 and 264
Duplicates found at indexes 342 and 280
Duplicates found at indexes 343 and 310
Duplicates found at indexes 364 and 58
Duplicates found at indexes 381 and 242
Duplicates found at indexes 385 and 32
Duplicates found at indexes 399 and 356
Duplicates found at indexes 433 and 423
Duplicates found at indexes 456 and 411
Duplicates found at indexes 473 and 438
Duplicates found at indexes 481 and 266
Duplicates found at indexes 483 and 181
Duplicates found at indexes 499 and 286
Duplicates found at indexes 511 and 372
Duplicates found at in

In [45]:
def find_missing_elements_and_indexes(longer_list, shorter_list):
    missing_elements = [element for element in longer_list if element not in shorter_list]
    missing_indexes = [longer_list.index(element) for element in missing_elements]
    return missing_elements, missing_indexes

missing_elements, missing_indexes = find_missing_elements_and_indexes(ids_that_went_in, ids_to_be_stored)
print("Missing elements:", missing_elements)
print("Missing element indexes:", missing_indexes)

Missing elements: []
Missing element indexes: []


In [ ]:
for idx in missing_indexes:
    resp = requests.get(song_data[idx][1], headers=headers)
    cured_resp = resp.json()
    cured_resp['_song_id'] = song_data[idx][0]
    tracks_audio_analysis.append(cured_resp)

In [ ]:
new_ids_that_went_in = [song_data[i][0] for i in range(len(song_data))]
new_ids_to_be_stored = [tracks_audio_analysis[i]['_song_id'] for i in range(len(tracks_audio_analysis))]

missing_elements, missing_indexes = find_missing_elements_and_indexes(new_ids_that_went_in, new_ids_to_be_stored)
print("Missing elements:", missing_elements)
print("Missing element indexes:", missing_indexes)

In [46]:
test = orig_dataset['id'].iloc[csv_split*(4)+4500]
test

'64riB16B0tLcZSwWrcF3y2'

In [47]:
test2 = orig_dataset['id'].iloc[-1]
test2

'6MAAMZImxcvYhRnxDLTufD'

In [48]:
check = [x['_song_id'] for x in tracks_audio_analysis ]
print("check 1:", check.index(test), "\ncheck 2:",check.index(test2))

check 1: 9 
check 2: 1792


In [49]:
dummy = []
dummy = pd.DataFrame(tracks_audio_analysis)

Since alot of the audio analysis data we wont be needing in this project as it would render the project too complicated, we will drop the columns that we dont need and use the most memory

In [50]:
memory_usage = dummy.memory_usage(deep=True)

# Sort the columns by memory usage in descending order
memory_usage_sorted = memory_usage.sort_values(ascending=False)

# Print the top columns by memory usage
print(memory_usage_sorted)

tatums      16844016
segments    12518576
beats        8492464
bars         2227088
track        2108240
meta          635360
sections      297840
_song_id      142595
Index            128
dtype: int64


Since we wont be needing the audio analysis of the sections but we could potentially use the number of sections in the song, we will be replacing the sections column in the dataset and replacing it with the length of it

In [30]:
bad_apple = []
for i in range(len(dummy)):
    try:
        a = len(dummy['sections'][i])
        #print(len(dummy['sections'][i]))
    except Exception as e:
        bad_apple.append(i)
        print(i,e)

In [ ]:
bad_apple

In [ ]:
dummy = dummy.drop(bad_apple)

In [51]:
dummy['no_of_sections'] = dummy['sections'].apply(lambda x: len(x))

In [52]:
to_drop = ['segments', 'tatums', 'beats', 'bars', 'sections']
dummy.drop(columns=to_drop, inplace=True)

Also in the track column, there are some memory expensive keys (codestring,echoprintstring,synchstring, and rhythmstring) that we want to remove

In [53]:
# Define a function to remove multiple keys from a dictionary
def remove_keys(dictionary, keys_to_remove):
    for key in keys_to_remove:
        if key in dictionary:
            del dictionary[key]
    return dictionary


# Specify the keys to remove
keys_to_remove = ['codestring', 'code_version', 'echoprintstring','echoprint_version',
                  'synchstring', 'synch_version', 'rhythmstring', 'rhythm_version'] 

# Apply the remove_keys function to the column
dummy['track'] = dummy['track'].apply(lambda x: remove_keys(x, keys_to_remove))

In [54]:
dummy['track'][0]

{'num_samples': 6615000,
 'duration': 300.0,
 'sample_md5': '',
 'offset_seconds': 0,
 'window_seconds': 0,
 'analysis_sample_rate': 22050,
 'analysis_channels': 1,
 'end_of_fade_in': 0.3658,
 'start_of_fade_out': 279.1619,
 'loudness': -5.531,
 'tempo': 150.067,
 'tempo_confidence': 0.335,
 'time_signature': 4,
 'time_signature_confidence': 0.546,
 'key': 4,
 'key_confidence': 0.2,
 'mode': 1,
 'mode_confidence': 0.37}

In [55]:
memory_usage = dummy.memory_usage(deep=True)

# Sort the columns by memory usage in descending order
memory_usage_sorted = memory_usage.sort_values(ascending=False)

# Print the top columns by memory usage
print(memory_usage_sorted)

track             2108240
meta               635360
_song_id           142595
no_of_sections      14440
Index                 128
dtype: int64


In [56]:
timestamp = time.strftime('%Y%m%d_%H%M%S')
dynamic_filename = f'track_audio_analysis{timestamp}.csv'
#Save DataFrame to the dynamic filename
dummy.to_csv(dynamic_filename, index=False)

<br>
combining the audio analysis csv files together and saving it

In [13]:
# Initialize an empty DataFrame to store the concatenated data
combined_df = pd.DataFrame()

# Loop through each CSV file and concatenate them
for i in range(1,9):
    df = pd.read_csv(f'track_audio_analysis_{i}.csv')
    combined_df = pd.concat([combined_df, df], ignore_index=True)

In [15]:
combined_df.to_csv('data/audio_analysis_dataset.csv', index= False)